In [1]:
!npm install -g localtunnel
!pip install flask-cors

import zipfile
import os
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import json
import threading
import time

app = Flask(__name__)
CORS(app)  # Permitir solicitudes CORS
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'OPTIONS'])
def home():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return "Server is running", 200

@app.route('/upload', methods=['POST', 'OPTIONS'])
def upload_file():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    if 'file' not in request.files or 'project_name' not in request.form or 'ud_name' not in request.form:
        return 'No file part, project name, or unit name', 400
    file = request.files['file']
    project_name = request.form['project_name']
    ud_name = request.form['ud_name']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        project_folder = os.path.join(app.config['UPLOAD_FOLDER'], project_name)
        ud_folder = os.path.join(project_folder, ud_name)
        os.makedirs(ud_folder, exist_ok=True)
        file_path = os.path.join(ud_folder, file.filename)
        file.save(file_path)
        process_uploaded_file(file_path, ud_folder)  # Llamar a la función para procesar el archivo
        return 'File uploaded and processed successfully', 200

@app.route('/check_auth', methods=['GET', 'OPTIONS'])
def check_auth():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return jsonify({"auth_status": "success"}), 200

def build_cors_preflight_response():
    response = jsonify({"message": "CORS preflight"})
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add("Access-Control-Allow-Headers", "*")
    response.headers.add("Access-Control-Allow-Methods", "GET, POST, OPTIONS, PUT, DELETE")
    return response

def process_uploaded_file(zip_path, extract_dir):
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Listar archivos extraídos
    extracted_files = os.listdir(extract_dir)
    print(extracted_files)

@app.route('/web/<path:path>')
def serve_file(path):
    return send_from_directory('/content/web', path)

@app.route('/modify', methods=['POST', 'OPTIONS'])
def modify_config():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    new_value = request.json.get('ntxCafCompressed')
    with open('/content/web/config.json', 'r') as config_file:
        config = json.load(config_file)

    config['ntxCafCompressed'] = new_value

    with open('/content/web/config.json', 'w') as config_file:
        json.dump(config, config_file)

    return jsonify({"status": "success"})

def run_app():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(5)  # Espera a que localtunnel se inicie

# Obtener la contraseña del túnel
tunnel_password = os.popen('curl https://loca.lt/mytunnelpassword').read().strip()
print(f"Tunnel password: {tunnel_password}")

!lt --port 5000


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.479s
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Tunnel password: 34.73.217.14
your url is: https://loose-apes-find.loca.lt


INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 11:42:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 11:42:20] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 11:42:41] "GET /check_auth HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 11:43:17] "POST /upload HTTP/1.1" 200 -


['trackingConfig.js', 'ud_files.zip', 'index.html', 'cmi5.json', 'components.json', '573_215_0_532', 'project.json']


...

['trackingConfig.js', 'ud_files.zip', 'index.html', 'cmi5.json', 'components.json', '573_215_0_532', 'project.json']


INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 11:45:17] "POST /upload HTTP/1.1" 200 -


['trackingConfig.js', 'ud_files.zip', 'index.html', 'cmi5.json', 'components.json', '573_215_0_532', 'project.json']


INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 11:45:49] "POST /upload HTTP/1.1" 200 -


['trackingConfig.js', 'ud_files.zip', 'index.html', 'cmi5.json', 'components.json', '573_215_0_532', 'project.json']
^C
